In [1]:
import pandas as pd
import numpy as np
from collections import Counter # для каждого элемента внутри массива считает количество повторений

In [2]:
# Загрузка данных и подготовка к обработке
df = pd.read_csv('Event_log.txt', sep='\t', encoding='cp1251')

In [4]:
# Функция, вычисляющая количество нарушений одним пользователем разграничения обязанностей в кейсе
def find_quantity_of_violations(incomp_events, activ):
    # Вход:  1)  Массив несовместимых событий
    #        2) События одного кейса в упорядоченном порядке 
    # Выход: Количество нарушений пользователя в одном кейсе
    
    num_of_violations = 0 # Количество нарушений в кейсе
    
    # Для каждого события подсчитываем  его количество повторений в кейсе
    sorted_col = Counter(activ)
    
    # Увеличение счетчика num_of_violations на число, равное количеству выполнений пользователем несовместимого события 
        num_of_violations += sorted_col[incomp_events[i]]
    return num_of_violations

# Функция, вычисляющая для каждого пользователя количество нарушений по всем кейсам и общее количество нарушений разграничения обязанностей в журнале событий
def SOD_anomalies(df):
    # Вход:  Журнал событий
    # Выход: Список из двух элементов. В первом элементе хранится список пользователей, отсортированный по количеству нарушений, а во втором общее количество нарушений
 
    # Запоминаем название события в incomp_event и соответствующие ему несовместимые события в incomp_with
    incomp_event1 = 'Заказ на поставку создан'
    incomp_event2 = 'Создание Заявки'
    incomp_event3 = 'Требование авансового платежа-Возврат'
    incomp_event4 = 'Счет заведен'
    incomp_with1 = ['Заказ на поставку согласован 1', 'Заказ на поставку согласован 2', 'Платеж (выравнивание)', 
                    'Заказ на поставку блокирован', 
                    'Заказ на поставку: изменен статус выпуска: ДанныеОтпр, возможны изменения',
                    'Заказ на поставку: изменен статус выпуска: ДанныеСогл, измен не возможны', 
                    'Заказ на поставку: изменен статус выпуска: НачКод, возможны изменения', 
                    'Заказ на поставку: изменен статус выпуска: Согл, измен не возможны']
    incomp_with2 = ['Заявка согласована']
    incomp_with3 = ['Авансовый платеж', 'Перерасчет авансового платежа', 'Перерасчет авансового платежа-Возврат', 
                    'Перерасчет по требованию авансового платежа']
    incomp_with4 = ['Счет блокирован: несоответствие цены', 'Счет предварительно полностью зарегистрирован', 
                    'Счет блокирован: несоответствие даты', 
                    'Счет предварительно зарегистрирован']
    
    # Группируем  колонку событий из входных данных  по  индентификатору  кейса и пользователю
    arrcase = df.groupby(['CaseID', 'User'])['Activity'].apply(list)
    arrcase = arrcase.reset_index()
    
    # Вспомогательные списки и переменные 
    users_violations_track = [] # Хранит списки из двух элементов. Первым элементом является пользователь, а вторым - количество его нарушений
    users = arrcase['User'].unique()
    users = list(users) # Список пользователей
    violations = 0
    total_violations = 0
    
    # Заполнение массива users_violations_track
    for i in range(len(users)):
        users_violations_track.append([users[i], 0])
        
    # Ищем нарушения разграничений обязанностей
    for i in range(len(arrcase['User'])):
        # Находим индекс пользователя в  списке пользователей
        user_index = users.index(arrcase['User'][i])
        
        # Проверяем выполнял ли пользователь событие, у которого есть список несовместимых событий
        if(incomp_event1 in arrcase['Activity'][i]):
            
            # Находим количество нарушений в текущем кейсе при помощи функции find_quantity_of_violations
            violations = find_quantity_of_violations(incomp_with1, arrcase['Activity'][i])
    
            #Обновление счетчиков
            users_violations_track[user_index][1] += violations # Увеличиваем счётчик нарушений для текущего пользователя
            total_violations += violations # Увеличиваем счетчик для общего количества нарушений
            
        # Далее, рассматриваем все события, у которых есть список несовместимых событий, аналогично трем предыдущим этапам
        if(incomp_event2 in arrcase['Activity'][i]):
            violations = find_quantity_of_violations(incomp_with2, arrcase['Activity'][i])
            users_violations_track[user_index][1] += violations
            total_violations += violations
        if(incomp_event3 in arrcase['Activity'][i]):
            violations = find_quantity_of_violations(incomp_with3, arrcase['Activity'][i])
            users_violations_track[user_index][1] += violations
            total_violations += violations 
        if(incomp_event4 in arrcase['Activity'][i]):
            violations = find_quantity_of_violations(incomp_with4, arrcase['Activity'][i])
            users_violations_track[user_index][1] += violations
            total_violations += violations
            
    # Сортируем пользовтелей в порядке убывания количества нарушений
    users_violations_track = sorted(users_violations_track, key = lambda x: x[1], reverse = True)
    
    # Отсекаем пользователей, у которых нету нарушений
    users_violations_track = list(filter(lambda x: x[1] != 0, users_violations_track))
    
    # Возвращаем список, первым элементом которого является отсортированный список пользователей нарушивших
    # разграничение обязанностей, а вторым  общее количество нарушений разграничения обязанностей
    return [users_violations_track, total_violations]